<a href="https://colab.research.google.com/github/antalvdb/mblm/blob/main/timbl_llm_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training a memory-based language model

MBLM offers an eco-friendly alternative to neural LLMs. MBLMs rely on CPUs; no GPUs or TPUs are required. Training MBLMs is costly in terms of RAM, but not in terms of time or computing resources. This notebook exemplifies how to train an MBLM model. MBLM comes in two flavors:

1.   **IB1**, k-Nearest Neighbor classification - accurate but slow and RAM-intensive;
2.   **IGTree**, decision-tree classification based on prefix tree retrieval; fast, compact, but less accurate.

This notebook assumes that you have uploaded a raw text file to Colab, or to Google Drive (moving the file to My Drive and mounting this drive). A sample text file can be downloaded from [here](https://antalvandenbosch.nl/mblm/edufineweb_train_000001-100k.txt). This file represents the first 100,000 lines of the first shard of the EduFineWeb refined web corpus by Hugging Face.

We start by installing the python bindings for TiMBL, the MBLM engine. We are also installing codecarbon to track CO2 emissions.

In [3]:
!apt install timbl
!pip install python3-timbl
!pip install codecarbon

import timbl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
timbl is already the newest version (6.5-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
timbl is already the newest version (6.5-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


We tokenize the training data with bert-base-cased, a Hugging Face tokenizer. This is the same tokenizer we will use for other data that will be handled by our model.

In [4]:
import os
import pandas as pd
from transformers import AutoTokenizer

def process_file(input_filename):
    # Generate the output filename by adding "tok" before the file extension
    base, ext = os.path.splitext(input_filename)
    output_filename = f"{base}_tok{ext}"

    # Read the input file
    with open(input_filename, 'r') as file:
        lines = file.readlines()

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['text'])

    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

    # Tokenize the text
    df['tokens'] = df['text'].apply(lambda x: tokenizer.tokenize(x))

    # Write the tokens to the output file
    with open(output_filename, 'w') as file:
        for tokens in df['tokens']:
            # Join the tokens list into a single string and write to the file
            file.write(' '.join(tokens) + '\n')

    print(f"Processed file saved as {output_filename}")

# Specify the filename directly
filename = "edufineweb_train_000001-100k.txt"  # Replace with your actual filename
input_filename = os.path.join('/content', filename)

# Process the file
process_file(input_filename)

Token indices sequence length is longer than the specified maximum sequence length for this model (585 > 512). Running this sequence through the model will result in indexing errors


Processed file saved as /content/edufineweb_train_000001-100k_tok.txt


We then create a file with fixed-width training instances that consist of a 16-word context as features, and the next word as the class label to be predicted.

In [5]:
import os

def generate_windowed_instances(file_path, output_file, window_size=16):
    # Start with an empty list to accumulate tokens for each block
    tokenized_text = []

    with open(file_path, 'r') as file, open(output_file, 'w') as outfile:
        for line in file:
            # Strip leading/trailing whitespace from the line
            stripped_line = line.strip()

            # Check if the line is empty, indicating the end of a block
            if not stripped_line:
                # Process the accumulated tokens for the current block
                if tokenized_text:
                    # Pad the beginning of the tokenized text with underscores
                    padded_text = ["_"] * window_size + tokenized_text

                    # Generate and print each windowed instance for this block
                    for i in range(window_size, len(padded_text) - 1):
                        context = padded_text[i - window_size:i]
                        target = padded_text[i]
                        outfile.write(f"{' '.join(context)} {target}\n")

                        # Reset tokenized_text for the next block
                        tokenized_text = []

            else:
                # Append tokens from the non-empty line to the current block
                tokenized_text.extend(stripped_line.split())

        # Process any remaining tokens after the last line
        if tokenized_text:
            padded_text = ["_"] * window_size + tokenized_text
            for i in range(window_size, len(padded_text) - 1):
                context = padded_text[i - window_size:i]
                target = padded_text[i]
                outfile.write(f"{' '.join(context)} {target}\n")

# Specify the input and output filenames directly
input_filename = "edufineweb_train_000001-100k_tok.txt"  # Replace with your actual input filename
output_filename = input_filename.replace(".txt", ".l16r0")  # Generate output filename

input_file_path = os.path.join('/content', input_filename)
output_file_path = os.path.join('/content', output_filename)

# Call the function to generate windowed instances and write to the output file
generate_windowed_instances(input_file_path, output_file_path)

Now we train our MBLM model with TiMBL. This can take a while and may consume high amounts of RAM.

The end result is `edufineweb_train_000001-100k_tok.l16r0.ibase`, an indexed and compressed instance base suitable for TiMBL classification. In LLM terms, this is the model file that you will need for your favorite LLM inference steps.

The option `-a0` means that the training set is compressed losslessly, with compression rates around 10-30%. This is the setting that implements **IB1**, k-Nearest Neighbor classification.

With `-a1`, a strong lossy compression is applied, yielding higher compression levels around 90-95%, and considerably faster but less accurate inference. This is TiMBL's **IGTree** option.

TiMBL is called from the Notebook shell commandline. It is wrapped inside a codecarbon CO2 emission measurement. TiMBL's quite verbose output is mixed with the equally verbose codecarbon information.



In [ ]:
from codecarbon import track_emissions

@track_emissions(project_name="mblm-edufineweb_train_000001-100k_tok.l16r0")
def train_model():
    !timbl -f edufineweb_train_000001-100k_tok.l16r0 -a0 +D -I edufineweb_train_000001-100k_tok.l16r0.ibase

train_model()

[codecarbon WARNING @ 15:43:40] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 15:43:40] [setup] RAM Tracking...
[codecarbon INFO @ 15:43:40] [setup] CPU Tracking...
[codecarbon WARNING @ 15:43:41] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 15:43:41] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 15:43:41] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 15:43:41] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:43:41] [setup] GPU Tracking...
[codecarbon INFO @ 15:43:41] No GPU found.
[codecarbon INFO @ 15:43:41] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

TiMBL 6.5 (c) CLST/ILK/CLIPS 1998 - 2020.
Tilburg Memory Based Learner
Centre for Language and Speech Technology, Radboud University
Induction of Linguistic Knowledge Research Group, Tilburg University
CLiPS Computational Linguistics Group, University of Antwerp
Thu May  8 15:43:41 2025

Examine datafile 'edufineweb_train_000001-100k_tok.l16r0' gave the following results:
Number of Features: 16
InputFormat       : Columns

Phase 1: Reading Datafile: edufineweb_train_000001-100k_tok.l16r0
Start:          0 @ Thu May  8 15:43:41 2025
Examining: 100000 @ Thu May  8 15:43:45 2025
Examining: 200000 @ Thu May  8 15:43:48 2025
Examining: 300000 @ Thu May  8 15:43:53 2025


[codecarbon INFO @ 15:43:56] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:43:56] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 15:43:56] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 15:43:56] 0.000219 kWh of electricity used since the beginning.


Examining: 400000 @ Thu May  8 15:43:57 2025
Examining: 500000 @ Thu May  8 15:44:02 2025
Examining: 600000 @ Thu May  8 15:44:07 2025


[codecarbon INFO @ 15:44:11] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:44:11] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 15:44:11] Energy consumed for All CPU : 0.000354 kWh
[codecarbon INFO @ 15:44:11] 0.000437 kWh of electricity used since the beginning.


...

Examining: 5000000 @ Thu May  8 15:48:13 2025
Finished:  5016728 @ Thu May  8 15:48:14 2025
Calculating Entropy         Thu May  8 15:48:14 2025
Lines of data     : 5016728


[codecarbon INFO @ 15:48:27] Energy consumed for RAM : 0.000791 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:48:27] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 15:48:27] Energy consumed for All CPU : 0.003365 kWh
[codecarbon INFO @ 15:48:27] 0.004156 kWh of electricity used since the beginning.


DB Entropy        : 10.739261
Number of Classes : 26915

Feats	Vals	InfoGain	GainRatio
    1  26915	2.1906111	0.20398148
    2  26915	2.1920201	0.20411270
    3  26915	2.1954564	0.20443265
    4  26915	2.1958033	0.20446497
    5  26915	2.2009414	0.20494344
    6  26915	2.2021521	0.20505616
    7  26915	2.2081435	0.20561406
    8  26915	2.2123939	0.20600985
    9  26915	2.2217645	0.20688242
   10  26915	2.2388949	0.20847754
   11  26915	2.2593096	0.21037848
   12  26915	2.2870857	0.21296491
   13  26915	2.3497464	0.21879964
   14  26915	2.4916539	0.23201355
   15  26915	2.7844700	0.25927949
   16  26915	3.7981671	0.35367119

Preparation took 286 seconds, 345 milliseconds and 785 microseconds
Feature Permutation based on GainRatio/Values :
< 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1 >
Phase 2: Building multi index on Datafile: edufineweb_train_000001-100k_tok.l16r0
Start:          0 @ Thu May  8 15:48:28 2025
Indexing:  100000 @ Thu May  8 15:48:30 2025
Indexing:  200000 @ Th

[codecarbon INFO @ 15:48:42] Energy consumed for RAM : 0.000833 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:48:42] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 15:48:42] Energy consumed for All CPU : 0.003542 kWh
[codecarbon INFO @ 15:48:42] 0.004375 kWh of electricity used since the beginning.


Indexing:  600000 @ Thu May  8 15:48:42 2025
Indexing:  700000 @ Thu May  8 15:48:44 2025
